In [15]:
# filelist
import cv2
import numpy as np
import os
from pathlib import Path
root = os.getcwd() # Don't change this
data_dirname = '/data_test/plantvillage/' # Change as you like 
p = Path(root + data_dirname)
# p.mkdir(exist_ok=True) 

#p = Path("C:/Users/mwels/Documents/Uni/11. Semester/Deep learning in visual recognition/Plant_leave_diseases_dataset_without_augmentation")
p.mkdir(exist_ok=True)


classes = [
  'Apple___Apple_scab',
  'Apple___healthy',
  'Apple___Black_rot',
  'Apple___Cedar_apple_rust',
  "all"]

if "all" in classes:
  classes = os.listdir(p)

for c in classes:
  print(c,end=" ")
  filelist = [x for x in (p/c).iterdir() if x.is_file()]
  for f in filelist:
    img = cv2.imread(str(f))
    if img is None:
      print(f'Failed to open {f}. Deleting file')
      os.remove(str(f))
# Pick first image of first class (i.e., cat)
# filelist = [x for x in (p/classes[0]).iterdir() if x.is_file()]
print(len(filelist))

Apple___Cedar_apple_rust Apple___Apple_scab Apple___Black_rot Apple___healthy 1645


In [16]:
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input

conv_base = MobileNet(weights='imagenet',
                      include_top=False,
                      input_shape=(40, 40, 3))
type(conv_base)

keras.engine.functional.Functional

In [32]:
from keras.layers import Dense,GlobalAveragePooling2D, Concatenate, Input, Flatten, Lambda
from keras.models import Model
# from src.fast_soft_sort.tf_ops import soft_rank


number_of_tiles = 4
image_net = conv_base
tiles = Input((4,40,40,3))
tile1 = Lambda(lambda x: x[:,0,:,:,:])(tiles) 
tile2 = Lambda(lambda x: x[:,1,:,:,:])(tiles) 
tile3 = Lambda(lambda x: x[:,2,:,:,:])(tiles) 
tile4 = Lambda(lambda x: x[:,3,:,:,:])(tiles) 
#tile5 = Lambda(lambda x: x[:,4,:,:,:])(tiles) 
#tile6 = Lambda(lambda x: x[:,5,:,:,:])(tiles) 
#tile7 = Lambda(lambda x: x[:,6,:,:,:])(tiles) 
#tile8 = Lambda(lambda x: x[:,7,:,:,:])(tiles) 
#tile9 = Lambda(lambda x: x[:,8,:,:,:])(tiles) 
tile1 = preprocess_input(tile1) 
tile2 = preprocess_input(tile2) 
tile3 = preprocess_input(tile3) 
tile4 = preprocess_input(tile4) 
#tile5 = preprocess_input(tile5) 
#tile6 = preprocess_input(tile6) 
#tile7 = preprocess_input(tile7) 
#tile8 = preprocess_input(tile8) 
#tile9 = preprocess_input(tile9) 

embedding_of_tile1 = image_net(tile1) 
embedding_of_tile2 = image_net(tile2)
embedding_of_tile3 = image_net(tile3) 
embedding_of_tile4 = image_net(tile4) 
#embedding_of_tile5 = image_net(tile5) 
#embedding_of_tile6 = image_net(tile6)
#embedding_of_tile7 = image_net(tile7)
#embedding_of_tile8 = image_net(tile8)
#embedding_of_tile9 = image_net(tile9)

concatonation = Concatenate(axis=3)([
    embedding_of_tile1,
    embedding_of_tile2,
    embedding_of_tile3,
    embedding_of_tile4])
    #embedding_of_tile5,
    #embedding_of_tile6,
    #embedding_of_tile7,
    #embedding_of_tile8,
    #embedding_of_tile9
    #])

out = Dense(500, activation="relu")(concatonation)
# out = Dense(500, activation="relu")(out)
out = Dense(24, activation="softmax")(out)
out = Flatten()(out)
# out = Lambda(lambda x: soft_rank(x, regularization_strength=0.5))(out)

model = Model(inputs=tiles, outputs=out)

In [33]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 4, 40, 40,   0           []                               
                                3)]                                                               
                                                                                                  
 lambda_49 (Lambda)             (None, 40, 40, 3)    0           ['input_10[0][0]']               
                                                                                                  
 lambda_50 (Lambda)             (None, 40, 40, 3)    0           ['input_10[0][0]']               
                                                                                                  
 lambda_51 (Lambda)             (None, 40, 40, 3)    0           ['input_10[0][0]']         

In [34]:
total_num_layers = len(model.layers)
num_base_layers = len(conv_base.layers)
print(f"Total number of layers is {total_num_layers}")
print(f"Number of pretrained base layers is {num_base_layers}")

for layer in model.layers[:num_base_layers]:
    layer.trainable=False
for layer in model.layers[num_base_layers:]:
    layer.trainable=True

Total number of layers is 18
Number of pretrained base layers is 86


In [35]:
from keras import optimizers
from sklearn.model_selection import train_test_split
from src.permutation_tools import *
import tensorflow_ranking as tfr 

x_train, x_test = train_test_split(filelist)

train_generator = PermNetDataGenerator(x_train, tilenumberx=2)
validation_generator = PermNetDataGenerator(x_test, tilenumberx=2)


# Set up optimizer
sgd_optimizer = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

# Compile model 
# softmax_loss = tfr.keras.losses.get(tfr.keras.losses.RankingLossKey.MEAN_SQUARED_LOSS)
model.compile(optimizer=sgd_optimizer,loss='categorical_crossentropy', metrics=['accuracy'])

step_size_train = train_generator.n//train_generator.batch_size # Number of mini-batches per epoch (training)
step_size_val = validation_generator.n//validation_generator.batch_size # Number of mini-batches per epoch (validation)

# Train model for 15 epochs
history = model.fit_generator(generator=train_generator,
                   validation_data=validation_generator,
                   validation_steps=step_size_val,
                   steps_per_epoch=step_size_train,
                   epochs=15)

/var/folders/l9/9d49h4t14s52dq6c276zwhgc0000gn/T/ipykernel_99466/2623075773.py:23: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_generator,


Epoch 1/15
19/19 [==============================] - 23s 811ms/step - loss: 3.2147 - accuracy: 0.0428 - val_loss: 3.1921 - val_accuracy: 0.0286
Epoch 2/15
19/19 [==============================] - 13s 699ms/step - loss: 3.2091 - accuracy: 0.0513 - val_loss: 3.2181 - val_accuracy: 0.0495
Epoch 3/15
19/19 [==============================] - 13s 689ms/step - loss: 3.2034 - accuracy: 0.0522 - val_loss: 3.2081 - val_accuracy: 0.0391
Epoch 4/15
19/19 [==============================] - 13s 699ms/step - loss: 3.2092 - accuracy: 0.0522 - val_loss: 3.1857 - val_accuracy: 0.0599
Epoch 5/15
19/19 [==============================] - ETA: 0s - loss: 3.2084 - accuracy: 0.0445

KeyboardInterrupt: 

4.6666665

In [ ]:
len(filelist)

630